### Установка PcapPlusPlus

In [ ]:
!wget https://github.com/seladb/PcapPlusPlus/releases/download/v22.11/pcapplusplus-22.11-ubuntu-20.04-gcc-9.tar.gz

In [ ]:
!mkdir pcpp_folder
!tar -xzvf "pcapplusplus-22.11-ubuntu-20.04-gcc-9.tar.gz" -C "pcpp_folder/"

In [ ]:
%cd pcpp_folder/pcapplusplus-22.11-ubuntu-20.04-gcc-9
%ls

In [ ]:
!./install.sh

In [ ]:
%cd ../..

In [ ]:
!apt install libpcap-dev

### Предобработка данных

In [ ]:
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/2yPkJS__Z5QZIA
!mv 2yPkJS__Z5QZIA common.pcap

In [ ]:
!rm -rf common_sessions_folder
!mkdir common_sessions_folder
!PcapSplitter -f common.pcap -o common_sessions_folder -m connection

In [ ]:
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/IVWwL_FLWKXXrQ
!mv IVWwL_FLWKXXrQ iot.pcap

In [ ]:
!rm -rf iot_sessions_folder
!mkdir iot_sessions_folder
!PcapSplitter -f iot.pcap -o iot_sessions_folder -m connection

##### Выделение признаков
scapy

In [ ]:
!pip install scapy

In [ ]:
from scapy.all import *
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
class Session(object):
  pass



---


Функции-экстракторы фичей идут сюда

Шаблон функции для экстракции признаков:
- принимает ссылку на объект session - в него нужно записать выделенный признак
- принимает scapy reader пакетов
- возвращает True - если всё окей, False - если сессия считается некорректной

 (например, если payload пустой)

In [ ]:
def sample_feature_extraction_function(session, reader):
  first_pkt = next(iter(reader))
  session.new_feature(first_pkt)
  return True

In [ ]:
MAX_PACKETS_TO_USE = 10

In [ ]:
def get_packet_length_features(session, reader):
  pkt_lens = []
  pld_lens = []
  i = 0
  for pkt in reader:
    # Что делать, если payload нулевой? Его не учитывать?
    pkt_lens.append(len(bytes(pkt)))
    pld_lens.append(len(bytes(pkt[IP].payload.payload)))
    i += 1
    if i == MAX_PACKETS_TO_USE:
      break
  pkt_n = len(pkt_lens)
  assert pkt_n >= 1
  session.pkt_n = pkt_n
  # Все, что не является average, а зависит от общего кол-ва пакетов в сессии
  # имеет не так много смысла из-за ограничения в 10 пакетов
  session.sum_pkt_lens = sum(pkt_lens)
  session.sum_pld_lens = sum(pld_lens)
  session.avg_pkt_len = session.sum_pkt_lens / pkt_n
  session.avg_pld_len = session.sum_pld_lens / pkt_n
  session.min_pkt_len = min(pkt_lens)
  session.max_pkt_len = max(pkt_lens)
  # среднеквадратичное отклонение
  if pkt_n == 1:
    session.std_dev_pkt_len = 0
  else:
    session.std_dev_pkt_len = (sum(((pkt_len - session.avg_pkt_len) ** 2 for pkt_len in pkt_lens)) / (pkt_n - 1)) ** 0.5
  return True

In [ ]:
def get_TTL(session, reader):
  i = 0
  sum_ttl = 0
  for pkt in reader:
    sum_ttl += pkt[IP].ttl
    i += 1
    if i == MAX_PACKETS_TO_USE:
      break
  assert i >= 1
  session.avg_ttl = sum_ttl / i
  return True

In [ ]:
def get_updownlink(session, subnet_ip_first_3):
  first_3_pieces = '.'.join(session.src.split('.')[0:3])
  session.updownlink = 0 if first_3_pieces == subnet_ip_first_3 else 1
  return True

In [ ]:
def get_session_duration(session, reader):
  start_time = None
  end_time = None
  i = 0
  for pkt in reader:
    i += 1
    if not start_time:
      start_time = pkt.time
    end_time = pkt.time
    if i == MAX_PACKETS_TO_USE:
      break
  # в секундах
  session.duration = end_time - start_time if end_time and start_time else 0
  return True

In [ ]:
def get_packet_delays(session, reader):
  min_delay = float('inf')
  max_delay = 0
  total_delay = 0
  num_packets = 0
  prev_time = None
  i = 0
  for pkt in reader:
    i += 1
    if not prev_time:
      prev_time = pkt.time
      continue
    delay = pkt.time - prev_time
    total_delay += delay
    num_packets += 1
    prev_time = pkt.time
    if delay > max_delay:
      max_delay = delay
    if delay < min_delay:
      min_delay = delay
    if i == MAX_PACKETS_TO_USE:
      break
  if num_packets > 0:
    session.avg_delay = total_delay / num_packets
  else:
    session.avg_delay = 0
  if num_packets == 0:
    session.min_delay = 0
  else:
    session.min_delay = min_delay
  session.max_delay = max_delay
  return True

In [ ]:
def get_rate_features(session):
  if (session.duration != 0):
    session.pkt_rate = session.pkt_n / session.duration
    session.bytes_rate = session.sum_pkt_lens / session.duration
  else:
    session.pkt_rate = 0
    session.bytes_rate = 0
  return True

---

In [ ]:
def extract_features(session, filepath, subnet_ip_first_3):
  return get_packet_length_features(session, PcapReader(filepath)) and \
  get_TTL(session, PcapReader(filepath)) and \
  get_updownlink(session, subnet_ip_first_3) and \
  get_session_duration(session, PcapReader(filepath)) and \
  get_packet_delays(session,PcapReader(filepath)) and \
  get_rate_features(session)

In [ ]:
def parse_sessions(folderpath, subnet_ip_first_3):
  onlyfiles = [f for f in listdir(folderpath) if isfile(join(folderpath, f))]
  bad_count = 0
  data = []
  for filename in tqdm(onlyfiles):
    filepath = join(folderpath, filename)
    session = Session()
    reader = PcapReader(filepath)
    pkt = next(iter(reader))
    try:
      session.src = pkt[IP].src
      session.dst = pkt[IP].dst
      session.sport = pkt[IP].sport
      session.dport = pkt[IP].dport
      session.l4_protocol = 0 if pkt[IP].payload.name == 'UDP' else 1
    except:
      bad_count += 1
      continue
    if extract_features(session, filepath, subnet_ip_first_3):
      data.append(session)
  print('%d sessions are bad' % bad_count)
  return data

In [ ]:
data_common = parse_sessions('common_sessions_folder', '192.168.2')
data_iot = parse_sessions('iot_sessions_folder', '192.168.1')

In [ ]:
class MyOneHot(object):
  def init(self, data):
    unique = np.unique(data)
    self.count = len(unique)
    self.mapper = dict(zip(unique, np.arange(0, self.count)))
    self.zeros = np.zeros(self.count)

  def transform(self, one_item):
    output = np.copy(self.zeros)
    if one_item in self.mapper:
      output[self.mapper[one_item]] = 1
    return output

In [ ]:
sport_one_hot = MyOneHot()
sport_one_hot.init([session.sport for session in data_common] + [session.sport for session in data_iot])

In [ ]:
dport_one_hot = MyOneHot()
dport_one_hot.init([session.dport for session in data_common] + [session.dport for session in data_iot])

In [ ]:
def numpy_features_from_data(data):
  return np.array(
    [np.concatenate((sport_one_hot.transform(session.sport),
                     dport_one_hot.transform(session.dport),
                     [
                     session.l4_protocol,
                     session.updownlink,
                     ],
                     [
                     session.pkt_n,
                     session.sum_pkt_lens,
                     session.sum_pld_lens,
                     session.avg_pkt_len,
                     session.avg_pld_len,
                     session.min_pkt_len,
                     session.max_pkt_len,
                     session.std_dev_pkt_len,
                     session.avg_ttl,
                     session.duration,
                     session.max_delay,
                     session.min_delay,
                     session.avg_delay,
                     session.pkt_rate,
                     session.bytes_rate,
                     ],
                     )) for session in data]
    , dtype='float32')

features_common = numpy_features_from_data(data_common)
features_iot = numpy_features_from_data(data_iot)

In [ ]:
print(features_common.shape)
print(features_iot.shape)

In [ ]:
X_features = np.append(features_common,
                       features_iot,
                       axis=0)

In [ ]:
y = np.append(np.zeros(features_common.shape[0]),
              np.ones(features_iot.shape[0]))

In [ ]:
def get_median_and_iqr(x):
  median = np.min(x, axis=0)
  iqr = np.max(x, axis=0) - np.min(x, axis=0)
  return median, iqr

In [ ]:
LAST_N_FEATURES_TO_NORMALIZE = 15
median, iqr = get_median_and_iqr(X_features[:, -LAST_N_FEATURES_TO_NORMALIZE:])
print(median, iqr)

In [ ]:
def normalize(x, median, iqr):
  x_normalized = x.copy()
  x_normalized[:, -LAST_N_FEATURES_TO_NORMALIZE:] = (x_normalized[:, -LAST_N_FEATURES_TO_NORMALIZE:] - median) / iqr
  return x_normalized

In [ ]:
X_features = normalize(X_features, median, iqr)

In [ ]:
print(X_features.shape)
print(y.shape)

### Обучение модели

In [ ]:
from sklearn.model_selection import train_test_split
X_features_train, X_features_test, y_train, y_test = train_test_split(X_features, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, features, targets):
    self.features = features
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):
    return self.features[idx], self.targets[idx]

In [ ]:
train_dataset = CustomDataset(X_features_train, y_train)
test_dataset = CustomDataset(X_features_test, y_test)

In [ ]:
weights = compute_class_weight('balanced', classes=[0, 1], y=y)
weights

SVM

In [ ]:
import torch.nn.functional as F

class TheSVM(nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(10484, 300)
    # self.dropout = nn.Dropout(p=0.8)
    self.fc2 = nn.Linear(300, 2)

  def forward(self, features):
    features = F.relu(self.fc1(features))
    # features = self.dropout(features)
    features = self.fc2(features)

    return features

Smoke test

In [ ]:
def smoke_test():
  model_classifier = TheSVM()
  features = torch.randn(1, 10484)
  assert model_classifier(features).shape == torch.Size([1, 2])
  print(model_classifier(features))
smoke_test()

Обучение

In [ ]:
def validate(model, val_loader, device):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for features, labels in val_loader:
      features = features.to(device)
      labels = labels.to(device)
      outputs = model(features)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  return correct / total

In [ ]:
def train(model):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  model.to(device)

  train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
  val_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

  criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(weights.astype('float32')).to(device))
  optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  for epoch in range(5):
    model.train()
    for features, labels in tqdm(train_loader):
      labels = labels.type(torch.LongTensor)
      optimizer.zero_grad()
      output = model(features.to(device))
      loss = criterion(output, labels.to(device))
      loss.backward()
      optimizer.step()
    accuracy = validate(model, val_loader, device)
    print("Epoch {} Loss {:.4f} Accuracy {:.4f}".format(epoch,loss.item(), accuracy))

In [ ]:
model_classifier = TheSVM()
train(model_classifier)
#

### Валидация модели

#### На моем трафике

In [ ]:
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/unZRnFR3X4IbeA
!mv unZRnFR3X4IbeA val_common.pcap

In [ ]:
!rm -rf val_common_sessions_folder
!mkdir val_common_sessions_folder
!PcapSplitter -f val_common.pcap -o val_common_sessions_folder -m connection

In [ ]:
class ValidationDataset(Dataset):
  def __init__(self, features):
    self.features = features

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx]

In [ ]:
data_val_common = parse_sessions('val_common_sessions_folder', '192.168.0')

In [ ]:
features_val_common = normalize(numpy_features_from_data(data_val_common), median, iqr)
val_common_dataset = ValidationDataset(features_val_common)

In [ ]:
def percent_of_iot(val_dataset):
  model_classifier.eval()
  iot_count = 0
  with torch.no_grad():
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    for features in tqdm(val_loader):
      output = model_classifier(features).argmax(dim=1)
      iot_count += torch.count_nonzero(output == 1)
  print('percent of iot traffic: %.2f%%' % (iot_count / len(val_dataset) * 100))

percent_of_iot(val_common_dataset)

#### На unsw acm sosr

In [ ]:
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/41qCNt-9M-Nk_g
!mv 41qCNt-9M-Nk_g val_iot.pcap

In [ ]:
!rm -rf val_iot_sessions_folder
!mkdir val_iot_sessions_folder
!PcapSplitter -f val_iot.pcap -o val_iot_sessions_folder -m connection

In [ ]:
import shutil
import os

def folder_into_parts(folderpath, n_parts):
  onlyfiles = [f for f in listdir(folderpath) if isfile(join(folderpath, f))]
  step = len(onlyfiles) // n_parts + (1 if len(onlyfiles) % n_parts != 0 else 0)
  part_i = 0
  for i in range(0, len(onlyfiles), step):
    part_i += 1
    newfolderpath = folderpath + '_part' + str(part_i)
    if os.path.exists(newfolderpath):
      shutil.rmtree(newfolderpath)
    os.mkdir(newfolderpath)
    for filename in onlyfiles[i:i+step]:
      shutil.copy(join(folderpath, filename), join(newfolderpath, filename))

In [ ]:
folder_into_parts('val_iot_sessions_folder', 13)

In [ ]:
data_val_iot = parse_sessions('val_iot_sessions_folder_part2', '192.168.1')

In [ ]:
features_val_iot = normalize(numpy_features_from_data(data_val_iot), median, iqr)
val_iot_dataset = ValidationDataset(features_val_iot)

In [ ]:
percent_of_iot(val_iot_dataset)

### Random Forest

В теории, для него очень плохо проводить one hot encoding, так как сильно повышается кол-во признаков

In [ ]:
def numpy_features_from_data_wo_one_hot(data):
  return np.array(
    [np.concatenate(([session.sport,
                     session.dport],
                     [
                     session.l4_protocol,
                     session.updownlink,
                     ],
                     [
                     session.pkt_n,
                     session.sum_pkt_lens,
                     session.sum_pld_lens,
                     session.avg_pkt_len,
                     session.avg_pld_len,
                     session.min_pkt_len,
                     session.max_pkt_len,
                     session.std_dev_pkt_len,
                     session.avg_ttl,
                     session.duration,
                     session.max_delay,
                     session.min_delay,
                     session.avg_delay,
                     session.pkt_rate,
                     session.bytes_rate,
                     ],
                     )) for session in data]
    , dtype='float32')

ski_features_common = numpy_features_from_data_wo_one_hot(data_common)
ski_features_iot = numpy_features_from_data_wo_one_hot(data_iot)

In [ ]:
ski_X_features = np.append(ski_features_common,
                           ski_features_iot,
                           axis=0)

In [ ]:
print(ski_X_features.shape)
print(y.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42, max_depth=10, class_weight='balanced')
clf.fit(ski_X_features, y)

In [ ]:
ski_features_val_common = numpy_features_from_data_wo_one_hot(data_val_common)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.zeros(len(ski_features_val_common)), clf.predict(ski_features_val_common))

In [ ]:
ski_features_val_iot = numpy_features_from_data_wo_one_hot(data_val_iot)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.ones(len(ski_features_val_iot)), clf.predict(ski_features_val_iot))

Просто интересно посмотреть на важность признаков

In [ ]:
feat_labels = [
    'sport',
    'dport',
    'l4_protocol',
    'updownlink',
    'pkt_n',
    'sum_pkt_lens',
    'sum_pld_lens',
    'avg_pkt_len',
    'avg_pld_len',
    'min_pkt_len',
    'max_pkt_len',
    'std_dev_pkt_len',
    'avg_ttl',
    'duration',
    'max_delay',
    'min_delay',
    'avg_delay',
    'pkt_rate',
    'bytes_rate',
    ]

In [ ]:
importances = clf.feature_importances_
sorted_indices = np.argsort(importances)[::-1]

In [ ]:
for feature_i in range(ski_features_val_iot.shape[1]):
    print("%2d) %-*s %f" % (feature_i + 1, 30,
                            feat_labels[sorted_indices[feature_i]],
                            importances[sorted_indices[feature_i]]))

 1) avg_ttl                        0.298940
 2) min_pkt_len                    0.123200
 3) max_delay                      0.120270
 4) duration                       0.088843
 5) avg_delay                      0.086342
 6) pkt_rate                       0.054872
 7) std_dev_pkt_len                0.035000
 8) bytes_rate                     0.030206
 9) sport                          0.026708
10) avg_pld_len                    0.026572
11) max_pkt_len                    0.023262
12) min_delay                      0.022541
13) avg_pkt_len                    0.021281
14) dport                          0.019432
15) sum_pld_lens                   0.011583
16) sum_pkt_lens                   0.008140
17) l4_protocol                    0.001319
18) pkt_n                          0.001101
19) updownlink                     0.000388
